<h1><center>Классификация c помощью векторных представлений ELMo</center></h1>

На этом занятии мы вернемся к задаче классификации текстов и построим модель классификации отзывов на фильмы на позитивный и негативный классы с помощью векторных представлений ELMo.

Мы снова будем использовать [датасет отзывов на фильмы IMDB](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews), с которым мы уже работали на предыдущих занятиях. Каждый из отзывов в датасете имеет свою оценку: является ли он позитивным или негативным.


## Подготовка данных



Установим  модули (torchvision и torchtext) и загрузим модель для токенизации:

In [1]:
!pip install torchvision

In [2]:
!pip install torchtext

Как мы делали и ранее, загрузим датасет IMDB из torchtext.

In [1]:
import spacy

In [2]:
#import en_core_web_sm
en_nlp = spacy.load('en_core_web_sm')

In [3]:
import numpy as np
import torch
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report,confusion_matrix
from torchtext import data
from torchtext import datasets
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter,defaultdict
%matplotlib inline
import seaborn as sns

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

train_src, test = datasets.IMDB.splits(TEXT, LABEL)

1.6.0


Разделим данные на обучаующую и тестовую выборку и преобразуем их в удобный нам в дальнейшем формат.

In [4]:
X_train = np.array([example.text for example in train_src])
y_train = np.array([0 if example.label == 'pos' else 1 for example in train_src])

X_test = np.array([example.text for example in test])
y_test = np.array([0 if example.label == 'pos' else 1 for example in test])


<ipython-input-4-365b897e045f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array([example.text for example in train_src])
<ipython-input-4-365b897e045f>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_test = np.array([example.text for example in test])


In [5]:
from collections import Counter

print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

train_counter = Counter(y_train)
test_counter = Counter(y_test)

print ("{0} positive and {1} negative examples in test".format(test_counter[1],test_counter[0]))
print ("{0} positive and {1} negative examples in test".format(test_counter[1],test_counter[0]))

total train examples 25000
total test examples 25000
12500 positive and 12500 negative examples in test
12500 positive and 12500 negative examples in test


Можно видеть, что классы сбалансированы: количество позитивных и негативных отзывов совпадают.

## Классификация с использованием векторных представлений ELMo

Напомним, что в прошлый раз мы использовали для классификации отзывов сверточную нейронную сеть, а в качестве векторных представлений слов брали предобученные вектора glove.

Теперь построим классификатор с помощью рекуррентной нейронной сети. Будем использовать для этого слой GRU.

Для получения векторных представлений текстов будем использовать предобученную модель ELMo с помощью библиотеки allennlp

In [4]:
!pip install allennlp

  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10179 sha256=829731a622cc81ed35700a64ea4ebbbf8973ea1d3d3913833947a80a4c7d2059
  Stored in directory: c:\users\basilio\appdata\local\pip\cache\wheels\6a\4f\72\28857f75625b263e2e3f5ab2fc4416c0a85960ac6485007eaa
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893262 sha256=f9cbf549c5bac404bb36955467b865aaecb79b842b9dd1ddea42be59e333e93a
  Stored in directory: c:\users\basilio\appdata\local\pip\cache\wheels\7b\78\f4\27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
Successfully built overrides sacremoses


Для удобства определим вспомогательный класс Data

In [6]:
from torch.utils.data import Dataset


class Data(Dataset):
    def __init__(self, x, y):
        self.data = list(zip(x, y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        assert idx < len(self)
        return self.data[idx]

Опишем класс *RNNEncoder*, который будет представлять собой основную часть модели классификации: на основе векторных представлений ELMo он будет применять рекурентный нейронный слой GRU. 

In [7]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class RNNEncoder(nn.Module):

    def __init__(self, input_size, hidden_size, out_size):
        '''
        input_size
        hidden_size: the output size of CNN/RNN/TR
        outpu_size: the final size of the encoder (after pooling)
        w
        CNN:
        - filters_num: feature_dim
        - filter_size: 3
        - pooling: max_pooling
        RNN:
        - hidden_size: feature_dim // 2
        - pooling: last hidden status
        Transformer
        - nhead: 2
        - nlayer: 1
        - pooling: average
        -------
        '''
        super(RNNEncoder, self).__init__()
        
        self.rnn = nn.GRU(input_size, hidden_size//2, batch_first=True, bidirectional=True)
        f_dim = hidden_size

        self.fc = nn.Linear(f_dim, out_size)
        nn.init.uniform_(self.fc.weight, -0.5, 0.5)
        nn.init.uniform_(self.fc.bias, -0.1, 0.1)

    def forward(self, inputs):
        out, _ = self.rnn(inputs)
        return self.fc(out.mean(1))

Опишем класс ElmoModel. Основной функционал класса: 
- для поступившего на вход текста получить его векторное представление с помощью модели ELMo 
- применить к получившимся векторам RNNEncoder
- применить линейный слой размерности  *batch_size* x *num_label* (для классификации)

Предобученная модель ELMo будет загружена при инициализации соответствующего класса *allennlp.modules.elmo*. Для этого нужно передать две ссылки $-$ ссылки на файлы с параметрами (*elmo_options_file*) и с весами (*elmo_weight_file*) модели. 



In [8]:
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
import numpy as np


class ElmoModel(nn.Module):
    def __init__(self,
                 use_gpu=True,
                 device="cuda:0",
                 out_size=64,
                 num_labels=2,
                 hidden_size=100,
                 dropout=0.5,
                 elmo_options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json",
                 elmo_weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5",
                 elmo_dim=512,):

        super(ElmoModel, self).__init__()
        self.use_gpu = use_gpu
        self.word_dim = elmo_dim
        self.device = device
        self.elmo_options_file = elmo_options_file
        self.elmo_weight_file = elmo_weight_file
        self.init_elmo()
        
        self.encoder = RNNEncoder(self.word_dim, hidden_size, out_size)
        self.cls = nn.Linear(out_size, num_labels)
        nn.init.uniform_(self.cls.weight, -0.1, 0.1)
        nn.init.uniform_(self.cls.bias, -0.1, 0.1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        word_embs = self.get_elmo(x)

        x = self.encoder(word_embs)
        x = self.dropout(x)
        x = self.cls(x)    # batch_size * num_label
        return x

    def init_elmo(self):
        '''
        initilize the ELMo model
        '''
        self.elmo = Elmo(self.elmo_options_file, self.elmo_weight_file, 1)
        for param in self.elmo.parameters():
            param.requires_grad = False
        
    def get_elmo(self, sentence_lists):
        '''
        get the ELMo word embedding vectors for a sentences
        '''
        character_ids = batch_to_ids(sentence_lists)
        if self.use_gpu:
            character_ids = character_ids.to(self.device)
        embeddings = self.elmo(character_ids)
        return embeddings['elmo_representations'][0]

Определим все необходимые для нашей модели параметры

In [9]:
# ELMo
elmo_options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
elmo_weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo_dim = 1024

hidden_size = 200
out_size = 64
num_labels = 2

use_gpu = False
seed = 2020
gpu_id = 0

dropout = 0.5
epochs = 15

test_size = 0.1
lr = 1e-3
weight_decay = 1e-4
batch_size = 8
device = "cpu"


Реализуем процедуру обучения нейронной сети

In [10]:
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

def now():
    return str(time.strftime('%Y-%m-%d %H:%M:%S'))


def collate_fn(batch):
    data, label = zip(*batch)
    return data, label

def train(x_train,
          x_test,
          y_train,
          y_test,
          seed=seed,
          use_gpu=True,
          batch_size=batch_size,
          test_size=test_size,
          lr=lr,
          epochs=epochs,
          weight_decay=weight_decay):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_gpu:
        torch.cuda.manual_seed_all(seed)


    train_data = Data(x_train, y_train)
    test_data = Data(x_test, y_test)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    model = ElmoModel(use_gpu=True,
                      device=device,
                      out_size=out_size,
                      num_labels=num_labels,
                      hidden_size=hidden_size,
                      dropout=0.5,
                      elmo_options_file=elmo_options_file,
                      elmo_weight_file=elmo_weight_file,
                      elmo_dim=elmo_dim)
    print(f"{now()} Elmo init model finished")

    if use_gpu:
        model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    lr_sheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.7)
    best_acc = -0.1
    best_epoch = -1
    start_time = time.time()
    for epoch in range(1, epochs):
        total_loss = 0.0
        model.train()
        for step, batch_data in enumerate(train_loader):
            # print(batch_data)
            x, labels = batch_data
            labels = torch.LongTensor(labels)
            if use_gpu:
                labels = labels.to(device)
            optimizer.zero_grad()
            # return model,x
            output = model(x)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        acc = test(model, test_loader)
        if acc > best_acc:
            best_acc = acc
            best_epoch = epoch
        print(f"{now()} Epoch{epoch}: loss: {total_loss}, test_acc: {acc}")
        lr_sheduler.step()

    end_time = time.time()
    print("*"*20)
    print(f"{now()} finished; epoch {best_epoch} best_acc: {best_acc}, time/epoch: {(end_time-start_time)/epochs}")


def test(model, test_loader):
    correct = 0
    num = 0
    model.eval()
    with torch.no_grad():
        for data in test_loader:
            x, labels = data
            num += len(labels)
            output = model(x)
            labels = torch.LongTensor(labels)
            if use_gpu:
                output = output.cpu()
            predict = torch.max(output.data, 1)[1]
            correct += (predict == labels).sum().item()
    model.train()
    return correct * 1.0 / num

In [11]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

In [ ]:
%%telegram_send ELMo_training_report
train(X_train,X_test,y_train,y_test, use_gpu=False)

2020-11-07 00:47:47 Elmo init model finished


Как видим, качество работы модели на этом же наборе данных выросло по сравнению с нейронной сетью CNN на векторах glove. Напомним, что максимальное качество классификации, которого удалось достичь тогда, было около 87%.

## Итоги
На этом занятии мы научились применять векторные представления ELMo в задаче бинарной классификации текстов.
Мы научились работать с библиотекой allennlp, чтобы получить предобученные вектора ELMo и использовали их для построение реккурентной нейронной сети (на основе GRU) для решения нужной задачи. 

Таким образом мы повысили качество классификации с 87% до 89% (по сравнению с моделью CNN на векторах glove).

На следующих занятиях мы познакомимся с моделями на основе архитектуры Transformer и научимся применять их в решении разных задач NLP.
